# **Sistema de colas de un solo servidor**



*   Los clientes llegan en intervalos,los cuales siguen un distribuccion uniforme.Asi mismo el tiempo de servicio en el servidor sigue dicha distribucion.
* El tiempo de llegada del proximo cliente se genera aleatoria al llegar una persona,de la misma manera.




In [28]:
import numpy as np
import pandas as pd

# Parametros

tiempo_llegada_max = 5
tiempo_llegada_min = 1

tiempo_service_max = 6
tiempo_service_min = 1

n = 200  #Numero eventos


In [29]:
np.random.seed(42)
event = 0

#inicio en ceros
time_ = 0
time_1 = 0
time_2 = 0

#Inicializamos en 0 para llegadas y atendidos
arrived_customers = 0
served_customers = 0
departed_customers = 0

#Generamos variables para siguiente evento
tiempo_llegada = np.random.uniform(tiempo_llegada_min, tiempo_llegada_max)
tiempo_llegada_siguiente = time_ + tiempo_llegada

server_status = "Inactivo"
cola = 0
arrived_customers += 1

# evento cero realizado
event += 1  


ts_columns = ['Evento', 'Time', 'type', 
              'cola', 'arr cust', 'served cust', 'depar cust']

time_series =  pd.DataFrame([[1, float(tiempo_llegada_siguiente), "Llegada", 
                              cola, arrived_customers, 0, 0]],
                             columns = ts_columns) 

time_series

,Evento,Time,type,cola,arr cust,served cust,depar cust
0,1,2.49816,Llegada,0,1,0,0


In [30]:
while event <= n:
    #Evento comienza
    event_type = time_series['type'].iloc[event-1]
    time_ = time_series['Time'].iloc[event-1]
    #Si el evento es una llegada
    if event_type == "Llegada": #Llegada genera por defecto la próxima hora de llegada
        #Clientes que llegaron aumenta a 1
        arrived_customers += 1
        
        #Genera siguiente hora de llegada
        tiempo_llegada = np.random.uniform(tiempo_llegada_min, tiempo_llegada_max)
        tiempo_llegada_siguiente = time_ + tiempo_llegada  
        #Si el cajero está inactivo se atiende al cliente de inmediato y genera tiempo de servicio

        if server_status == "Inactivo":
            #Cliente en servicio aumenta en 1
            served_customers += 1
            #Este número de cliente se agrega a la columna 'cliente atendido' en el evento n
            time_series['served cust'].iloc[event-1] =  served_customers

            #Genera evento siguiente tiempo de servicio y de salida
            tiempo_service = np.random.uniform(tiempo_service_min, tiempo_service_max)
            tiempo_salida = time_ + tiempo_service
            departed_customers += 1 #mismo cliente que se atiende a la hora de llegada sale a la hora de salida

            #Eventos generados agregados a la serie de tiempo
            generated_events =  pd.DataFrame([
                          [99, float(tiempo_salida), "Salida", 0, 0,0,  departed_customers],
                          [99, float(tiempo_llegada_siguiente), "Llegada", 0, arrived_customers, 0, 0]
                          ], columns = ts_columns) 
                          #Se ordena después
            time_series =  pd.concat([time_series, generated_events])
            #Se ordena por tiempo los eventos 
            time_series = time_series.sort_values(['Time'])
            time_series.reset_index(drop=True, inplace=True)
            #Número de evento se asigna en orden del tiempo
            time_series['Evento'] = list(range(1, time_series.shape[0]+1))

            #Evento finaliza
            event += 1 

        #Si el estado es ocupado, aumenta cola y solo genera llegada
        if server_status == "Ocupado":
            cola += 1
            #Eventos generados agregados a la serie de tiempo
            generated_events =  pd.DataFrame([
                                [99, float(tiempo_llegada_siguiente), "Llegada", 
                                 0, arrived_customers,0, 0]]
                                , columns = ts_columns) 

            time_series =  pd.concat([time_series, generated_events])
            time_series = time_series.sort_values(['Time'])
            time_series.reset_index(drop=True, inplace=True)
            time_series['Evento'] = list(range(1, time_series.shape[0]+1))
            time_series['cola'].iloc[event-1] = cola
            #Evento finaliza
            event += 1 
    #Si el evento es una salida
    if event_type == "Salida":
        #si la cola es cero y el cliente sale, el estado del servidor permanece inactivo y el próximo evento es una llegada
        if cola == 0 :
            server_status = "Inactivo"
            #Evento finaliza/
            #No sucede nada hasta proxima llegada
            event += 1

        #si hay clientes en cola (>0), el servidor cambia a ocupado y la cola disminuye en uno   
        if cola != 0 :
            #Cliente es atendido y el contador de clientes atendidos aumenta en 1
            served_customers += 1
            
            #este número de cliente se agrega a la columna 'cliente atendido' en el evento n
            time_series['served cust'].iloc[event-1] =  served_customers           
            
            #Cola disminuye en 1
            cola -= 1
            server_status = "Ocupado"
            
            #Genera evento siguiente tiempo de servicio y de salida
            tiempo_service = np.random.uniform(tiempo_service_min, tiempo_service_max)
            tiempo_salida = time_ + tiempo_service
            departed_customers += 1 #mismo cliente que se atiende a la hora de llegada sale a la hora de salida
            #Eventos generados agregados a la serie de tiempo
            generated_events =  pd.DataFrame([
                                    [99, float(tiempo_salida), "Salida", 0, 0, 0, departed_customers]
                                    ], columns = ts_columns) 
            time_series =  pd.concat([time_series, generated_events])
            time_series = time_series.sort_values(['Time'])
            time_series.reset_index(drop=True, inplace=True)
            time_series['Evento'] = list(range(1, time_series.shape[0]+1)) 
            time_series['cola'].iloc[event-1] = cola
            #Evento finaliza
            event += 1 

    #Finalizado el evento, se determina el estado del cajero
    #Si es la próxima llegada si es anterior a la salida del cliente actual, el servidor estará ocupado a la llegada
    if tiempo_llegada_siguiente < tiempo_salida:
        server_status = "Ocupado" 
    else: 
        server_status = "Inactivo"


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [31]:
#create summary of customer data with results
arrivals = time_series.loc[time_series['type'] == 'Llegada', ['Time', 'arr cust' ]]
arrivals.columns = ['time', 'customer']
depature = time_series.loc[time_series['type'] == 'Salida', ['Time', 'depar cust' ]]
depature.columns = ['time', 'customer']

serving = time_series.loc[time_series['served cust'] != 0 , ['Time', 'served cust' ]]
serving.columns = ['Time', 'customer']

customer_df = arrivals.merge(depature, on='customer')
customer_df = customer_df.merge(serving, on='customer')
customer_df.columns = ['tiempo llegada', 'cliente', 'tiempo Salida', 'tiempo Ingreso']
customer_df = customer_df[['cliente', 'tiempo llegada', 'tiempo Ingreso', 'tiempo Salida']] 

customer_df['tiempo en cola'] = customer_df['tiempo Ingreso'] - customer_df['tiempo llegada'] 

customer_df['duracion de servicio'] = customer_df['tiempo Salida'] - customer_df['tiempo Ingreso'] 
customer_df = customer_df.round(2)
customer_df.head(16)

,cliente,tiempo llegada,tiempo Ingreso,tiempo Salida,tiempo en cola,duracion de servicio
0,1,2.50,2.50,7.16,0.00,4.66
1,2,7.30,7.30,9.08,0.00,1.78
2,3,10.70,10.70,11.99,0.00,1.29
3,4,12.32,12.32,16.33,0.00,4.01
4,5,16.78,16.78,17.89,0.00,1.10
5,6,20.62,20.62,25.78,0.00,5.16
6,7,25.50,25.78,27.69,0.28,1.91
7,8,27.35,27.69,30.21,0.34,2.52
8,9,29.08,30.21,33.37,1.13,3.16
9,10,32.18,33.37,37.43,1.19,4.06


In [32]:
np.random.seed(42)
event = 0

#inicio en ceros
time_ = customer_df['tiempo Salida'].get(0)


#Inicializamos en 0 para llegadas y atendidos
arrived_customers = 0
served_customers = 0
departed_customers = 0

#Generamos variables para siguiente evento
tiempo_llegada = np.random.uniform(0, 1)
tiempo_llegada_siguiente = time_ + tiempo_llegada

server_status = "Inactivo"
cola = 0
arrived_customers += 1

# evento cero realizado
event += 1  


ts_columns = ['Evento', 'Time', 'type', 
              'cola', 'arr cust', 'served cust', 'depar cust']

time_series =  pd.DataFrame([[1, float(tiempo_llegada_siguiente), "Llegada", 
                              cola, arrived_customers, 0, 0]],
                             columns = ts_columns) 


In [33]:
while event <= customer_df['tiempo Salida'].shape[0]:

    #Evento comienza
    event_type = time_series['type'].iloc[event-1]
    time_ = customer_df['tiempo Salida'].get(event-1)
    
    #Si el evento es una llegada
    if event_type == "Llegada": #Llegada genera por defecto la próxima hora de llegada
        #Clientes que llegaron aumenta a 1
        arrived_customers += 1
        
        #Genera siguiente hora de llegada
        tiempo_llegada = np.random.uniform(tiempo_llegada_min, tiempo_llegada_max)
        tiempo_llegada_siguiente = time_ + tiempo_llegada  
        #Si el cajero está inactivo se atiende al cliente de inmediato y genera tiempo de servicio

        if server_status == "Inactivo":
            #Cliente en servicio aumenta en 1
            served_customers += 1
            #Este número de cliente se agrega a la columna 'cliente atendido' en el evento n
            time_series['served cust'].iloc[event-1] =  served_customers

            #Genera evento siguiente tiempo de servicio y de salida
            tiempo_service = np.random.uniform(tiempo_service_min, tiempo_service_max)
            tiempo_salida = time_ + tiempo_service
            departed_customers += 1 #mismo cliente que se atiende a la hora de llegada sale a la hora de salida

            #Eventos generados agregados a la serie de tiempo
            generated_events =  pd.DataFrame([
                          [99, float(tiempo_salida), "Salida", 0, 0,0,  departed_customers],
                          [99, float(tiempo_llegada_siguiente), "Llegada", 0, arrived_customers, 0, 0]
                          ], columns = ts_columns) 
                          #Se ordena después
            time_series =  pd.concat([time_series, generated_events])
            #Se ordena por tiempo los eventos 
            time_series = time_series.sort_values(['Time'])
            time_series.reset_index(drop=True, inplace=True)
            #Número de evento se asigna en orden del tiempo
            time_series['Evento'] = list(range(1, time_series.shape[0]+1))

            #Evento finaliza
            event += 1 

        #Si el estado es ocupado, aumenta cola y solo genera llegada
        if server_status == "Ocupado":
            cola += 1
            #Eventos generados agregados a la serie de tiempo
            generated_events =  pd.DataFrame([
                                [99, float(tiempo_llegada_siguiente), "Llegada", 
                                 0, arrived_customers,0, 0]]
                                , columns = ts_columns) 

            time_series =  pd.concat([time_series, generated_events])
            time_series = time_series.sort_values(['Time'])
            time_series.reset_index(drop=True, inplace=True)
            time_series['Evento'] = list(range(1, time_series.shape[0]+1))
            time_series['cola'].iloc[event-1] = cola
            #Evento finaliza
            event += 1 
    #Si el evento es una salida
    if event_type == "Salida":
        #si la cola es cero y el cliente sale, el estado del servidor permanece inactivo y el próximo evento es una llegada
        if cola == 0 :
            server_status = "Inactivo"
            #Evento finaliza/
            #No sucede nada hasta proxima llegada
            event += 1

        #si hay clientes en cola (>0), el servidor cambia a ocupado y la cola disminuye en uno   
        if cola != 0 :
            #Cliente es atendido y el contador de clientes atendidos aumenta en 1
            served_customers += 1
            
            #este número de cliente se agrega a la columna 'cliente atendido' en el evento n
            time_series['served cust'].iloc[event-1] =  served_customers           
            
            #Cola disminuye en 1
            cola -= 1
            server_status = "Ocupado"
            
            #Genera evento siguiente tiempo de servicio y de salida
            tiempo_service = np.random.uniform(tiempo_service_min, tiempo_service_max)
            tiempo_salida = time_ + tiempo_service
            departed_customers += 1 #mismo cliente que se atiende a la hora de llegada sale a la hora de salida
            #Eventos generados agregados a la serie de tiempo
            generated_events =  pd.DataFrame([
                                    [99, float(tiempo_salida), "Salida", 0, 0, 0, departed_customers]
                                    ], columns = ts_columns) 
            time_series =  pd.concat([time_series, generated_events])
            time_series = time_series.sort_values(['Time'])
            time_series.reset_index(drop=True, inplace=True)
            time_series['Evento'] = list(range(1, time_series.shape[0]+1)) 
            time_series['cola'].iloc[event-1] = cola
            #Evento finaliza
            event += 1 

    #Finalizado el evento, se determina el estado del cajero
    #Si es la próxima llegada si es anterior a la salida del cliente actual, el servidor estará ocupado a la llegada
    if tiempo_llegada_siguiente < tiempo_salida:
        server_status = "Ocupado" 
    else: 
        server_status = "Inactivo"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [34]:
arrivals = time_series.loc[time_series['type'] == 'Llegada', ['Time', 'arr cust' ]]
arrivals.columns = ['time', 'customer']
depature = time_series.loc[time_series['type'] == 'Salida', ['Time', 'depar cust' ]]
depature.columns = ['time', 'customer']

serving = time_series.loc[time_series['served cust'] != 0 , ['Time', 'served cust' ]]
serving.columns = ['Time', 'customer']

customer_df = arrivals.merge(depature, on='customer')
customer_df = customer_df.merge(serving, on='customer')
customer_df.columns = ['tiempo llegada', 'cliente', 'tiempo Salida', 'tiempo Ingreso']
customer_df = customer_df[['cliente', 'tiempo llegada', 'tiempo Ingreso', 'tiempo Salida']] 

customer_df['tiempo en cola'] = customer_df['tiempo Ingreso'] - customer_df['tiempo llegada'] 

customer_df['duracion de servicio'] = customer_df['tiempo Salida'] - customer_df['tiempo Ingreso'] 
customer_df = customer_df.round(2)
customer_df.head(16)

,cliente,tiempo llegada,tiempo Ingreso,tiempo Salida,tiempo en cola,duracion de servicio
0,1,7.53,7.53,11.82,0.00,4.29
1,2,11.96,11.96,13.77,0.00,1.81
2,3,15.38,15.38,19.18,0.00,3.80
3,4,19.51,19.51,31.70,0.00,12.18
4,5,32.15,32.15,34.47,0.00,2.32
5,6,37.20,37.20,45.42,0.00,8.22
6,7,45.14,45.42,51.06,0.28,5.64
7,8,47.04,51.06,59.26,4.02,8.20
8,9,52.73,59.26,63.92,6.53,4.66
9,10,62.37,63.92,69.63,1.55,5.71
